<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/good_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install exchange_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler


In [4]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [12]:

class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99]):
      self.period = period
      self.increased = increased
      self.traded = traded

      self.price_dic = {}

      self.start, self.today = self.get_date()
      self.df = self.get_stock_df()
      self.filtered_df = self.get_filtered_df()
      self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_position(self, df):
    return df['Close'][-1]/df['Close'].max()

  def get_result_df(self):
    for i in self.filtered_df['종목코드']:
      self.price_dic[i] = fdr.DataReader(str(i), self.start, self.today)

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Position'] = [self.get_position(self.price_dic[x]) for x in self.price_dic]

    t = self.filtered_df.sort_values('Sharpe', ascending=False)

    scaler = MinMaxScaler(feature_range=(5, 10))

    t['Position'] = scaler.fit_transform(t[['Position']]).round(1)
    t['Sortino'] = scaler.fit_transform(t[['Sortino']]).round(1)
    t['Sharpe'] = scaler.fit_transform(t[['Sharpe']]).round(1)
    t['거래대금'] = scaler.fit_transform(t[['거래대금']]).round(1)
    t['등락률'] = scaler.fit_transform(t[['등락률']]).round(1)
    t['합산'] = t['Position'] + t['Sortino'] + t['Sharpe'] + t['거래대금'] + t['등락률']
    t['합산'] = scaler.fit_transform(t[['합산']]).round(1)

    t = t[['종목명', '종료일 종가', '등락률', '거래대금',	'Sharpe',	'Sortino',	'Position', '합산']].reset_index(drop=True).rename({'종료일 종가':'현재가', '등락률':'파워', '거래대금':'관심도', 'Sharpe':'Risk1', 'Sortino':'Risk2', 'Position':'모멘텀'}, axis=1)
    t.index+=1
    t.index.name='순위'

    return t

  def plot_df(self, start_from=0, to=5):
    fig = go.Figure()
    for i in self.price_dic:
      if (i in self.result_df['종목코드'][start_from:to].values):
        fig.add_trace(go.Scatter(x= self.price_dic[i].index, \
                                 y= self.price_dic[i]['Close']/self.price_dic[i]['Close'][0], \
                                 name= self.filtered_df[self.filtered_df['종목코드']==i]['종목명'].item()))

    fig.update_layout(title=self.today,
                      xaxis_title='Date',
                      yaxis_title='Return (%)')
    fig.show()

  def plot_stock(self, id):
    t = stocklist.price_dic[id]
    candlesticks = go.Candlestick(
        x=t.index,
        open=t['Open'],
        high=t['High'],
        low=t['Low'],
        close=t['Close'],
        showlegend=False
    )

    volume_bars = go.Bar(
        x=t.index,
        y=t['Volume'],
        showlegend=False,
        marker={
            "color": "rgba(128,128,128,0.5)",
        }
    )

    fig = go.Figure(candlesticks)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(candlesticks, secondary_y=False
                  )
    fig.add_trace(volume_bars, secondary_y=True)
    fig.update_layout(title= self.filtered_df[self.filtered_df['종목코드']==id]['종목명'].item(), height=800)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        ]
    )
    fig.update_yaxes(title="Price $", secondary_y=True, showgrid=True)
    fig.update_yaxes(title="Volume $", secondary_y=False, showgrid=False)
    fig.show()

In [13]:
highest = [0.98, 1]
higher = [0.9, 0.99]
longer = 500

In [14]:
stocklist = StockList()

In [15]:
t = stocklist.result_df
t

,종목명,현재가,파워,관심도,Risk1,Risk2,모멘텀,합산
순위,,,,,,,,
1,에스앤에스텍,49000,7.1,5.1,10.0,9.6,10.0,9.9
2,메디톡스,248000,6.6,5.6,9.7,9.8,9.2,9.6
3,포스코DX,13190,6.2,8.3,8.9,10.0,7.8,9.7
4,에코플라스틱,7150,8.3,6.5,8.1,9.1,10.0,10.0
5,바이오니아,73200,7.7,7.2,8.1,8.2,9.9,9.7
6,삼아알미늄,71600,9.1,5.8,8.0,8.7,7.4,8.8
7,레이크머티리얼즈,15370,9.7,7.3,7.9,7.2,9.5,9.8
8,코스모화학,57600,8.3,10.0,7.4,7.7,6.5,9.2
9,애경케미칼,18170,5.0,5.5,7.4,6.3,9.2,6.7


In [16]:
from google.colab import files
t.to_csv('ai_df.csv', encoding="utf-8-sig") 
files.download('ai_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
stocklist = StockList(250, highest, highest)
t = stocklist.result_df

In [18]:
from google.colab import files
t.to_csv('king_df.csv', encoding="utf-8-sig") 
files.download('king_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>